In [9]:
# Cargar librerías
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
import mlflow
import mlflow.sklearn


In [10]:
# Leer datos
df = pd.read_csv("data/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
# identifica las columnas que tiene valores 0
(df==0).sum()

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

In [12]:
# Reemplazar ceros en columnas específicas por NaN
cols_to_clean = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin']
# No se consideran algunas, por ejemplo Pregnancies (por qué si es posible que se tengan cero embarazos)

# Reemplazar ceros por NaN para poder tratarlos como datos faltantes
for col in cols_to_clean:
    df[col] = df[col].replace(0, np.nan)

# Reemplazar NaN con la moda (valor más frecuente) de cada columna
for col in cols_to_clean:
    moda = df[col].mode()[0]# buscar el método más común para determinar la moda de una columna
    df[col] = df[col].fillna(moda)


In [13]:
# Separar datos
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
# agregar el traking uri de mlflow
mlflow.set_tracking_uri("http://127.0.0.1:9090")
mlflow.set_experiment(experiment_name="ClasificadorDemoDiabetes")
# agregar set_experiment con nombre: ClasificadorDemoDiabetes


2025/05/17 06:56:12 INFO mlflow.tracking.fluent: Experiment with name 'ClasificadorDemoDiabetes' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1747482972081, experiment_id='1', last_update_time=1747482972081, lifecycle_stage='active', name='ClasificadorDemoDiabetes', tags={}>

In [15]:
# Entrenamiento y registro con MLflow
C = 1.0
max_iter = 1000
# usar los parámetros del archivo logreg_variaciones_educativas.csv
# para los parámtros C, max_iter, solver y penalty de LogisticRegresion
# se debe genear un run por cada fila del archivo, usando sus parámetros
# Analizar que usar

# agregar la línea del start_run
data = pd.read_csv("data/logreg_variaciones_educativas.csv", header=0)
c = data['logreg_C'].tolist()
m = data['logreg_max_iter'].tolist()
solver = data['solver'].tolist()
penalty = data['penalty'].tolist()

for i in range(len(c)):

    with mlflow.start_run(run_name=f"LR_Run_{i}"):
        # El pipeline usar StandarScaler para que todos los valore numéricos estén en la misma escala
        # no cambiar dicha línea
        # LogisticRegresion deben variar sus parámetros en cada iteración.
        pipeline = Pipeline([
                ("scaler", StandardScaler()),
                ("clf", LogisticRegression(
                    C=c[i],
                    max_iter=m[i],
                    solver=solver[i],
                    penalty=penalty[i]
                ))
            ])

        # Entrenar y evaluar
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred)

        # recuerde que log_param y log_metric debe ir guardando los valores
        # dependiendo del run correspondiente de cada fila
        mlflow.log_param("logreg_C", C)
        mlflow.log_param("logreg_max_iter", max_iter)
        mlflow.log_param("solver", "liblinear")
        mlflow.log_param("penalty", "l2")

        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)

        # Guardar el pipeline completo
        mlflow.sklearn.log_model(pipeline, "modelo_pipeline")

        print(" Modelo registrado en MLflow")
        print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f}")


2025/05/17 06:56:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


 Modelo registrado en MLflow
Accuracy: 0.7208 | Precision: 0.6034
🏃 View run LR_Run_0 at: http://127.0.0.1:9090/#/experiments/1/runs/dc53d79df68b4a9aa688f4d170c0afba
🧪 View experiment at: http://127.0.0.1:9090/#/experiments/1


2025/05/17 06:56:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
🏃 View run LR_Run_1 at: http://127.0.0.1:9090/#/experiments/1/runs/7514aa98fe524b558e663d5569252035
🧪 View experiment at: http://127.0.0.1:9090/#/experiments/1


2025/05/17 06:56:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
🏃 View run LR_Run_7 at: http://127.0.0.1:9090/#/experiments/1/runs/97955305de454b509e8a8d04df20e4f0
🧪 View experiment at: http://127.0.0.1:9090/#/experiments/1


2025/05/17 06:56:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
🏃 View run LR_Run_8 at: http://127.0.0.1:9090/#/experiments/1/runs/7ae2b111d92c40879af7137d3c9435ce
🧪 View experiment at: http://127.0.0.1:9090/#/experiments/1


2025/05/17 06:56:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
🏃 View run LR_Run_9 at: http://127.0.0.1:9090/#/experiments/1/runs/eed43156273b4c2fbbac1199f2a987b3
🧪 View experiment at: http://127.0.0.1:9090/#/experiments/1
